<a href="https://colab.research.google.com/github/Murilo1514/Diabets_prediction_pipeline/blob/main/SVM_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna
!pip install ucimlrepo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import optuna
from plotly.io import show
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, FunctionTransformer, RobustScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score
from sklearn.compose import ColumnTransformer
from ucimlrepo import fetch_ucirepo
from functools import partial
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'optuna'

In [ ]:
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

df = cdc_diabetes_health_indicators.data.features
aux = cdc_diabetes_health_indicators.data.targets

df["diabetes"] = aux["Diabetes_binary"]

# df = df.sample(frac=0.01, random_state=42)

print(cdc_diabetes_health_indicators.metadata)

print(cdc_diabetes_health_indicators.variables)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

<ipython-input-24-4317efe546fa>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df.duplicated().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



24206

In [ ]:
df.drop_duplicates(inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

<ipython-input-26-16cdb8520be8>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
X = df.drop(columns=["diabetes"])
y = df["diabetes"]

x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)

categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']
binary_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']


#print(y_train.value_counts())
#print(y_test.value_counts())

x_validation, x_test, y_validation, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=10, stratify=y_temp)

print("Treino\n",y_train.value_counts())
print("\nValidação \n",y_validation.value_counts())
print("\nTeste\n",y_test.value_counts())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Treino
 diabetes
0    155501
1     28078
Name: count, dtype: int64

Validação 
 diabetes
0    19438
1     3509
Name: count, dtype: int64

Teste
 diabetes
0    19438
1     3510
Name: count, dtype: int64


In [ ]:
preprocessor_svm = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns),
        ('bin', 'passthrough', binary_columns)
    ])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
pca = PCA(n_components=5)
pca.fit(x_train)
X_train = pca.fit_transform(x_train)
X_test = pca.fit_transform(x_test)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
X_train.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,0,1,2,3,4
0,0.563583,-11.738621,2.028129,-4.866915,0.387328
1,16.690553,-4.766932,14.951749,-1.162146,-3.076461
2,5.514144,0.257385,-8.944183,1.636671,1.956569
3,-1.459759,-2.387550,1.384837,4.489885,-1.794192
4,-5.978750,0.341081,-0.662873,2.717967,1.559649


In [ ]:
def objective(trial):
    pipeline = Pipeline([
        #('outlier_remover', outlier_remover),
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])


    C = trial.suggest_float('C', 0.001, 1000, log =True)
    # suggest_float(..., log=True)
    gamma = trial.suggest_float('gamma', 0.0001, 10, log = True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])

    # print(C)

    pipeline.set_params(svm__C = C, svm__gamma = gamma, svm__kernel = kernel)
    pipeline.fit(x_train, y_train)

    y_pred = pipeline.predict(x_validation)
    score = f1_score(y_validation, y_pred)
    return score


#objective_func = partial(objective, pipeline=pipe_knn)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1) # incrementação do parâmetro n_jobs=-1
# study.optimize(objective_func, n_trials=100)

study.best_params

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

[I 2024-11-17 21:26:31,040] A new study created in memory with name: no-name-8123c1ac-a287-4ea3-8673-128953bf4010
[I 2024-11-17 21:27:06,124] Trial 17 finished with value: 0.003403289846851957 and parameters: {'C': 87.56458421084405, 'gamma': 0.2983967710636374, 'kernel': 'linear'}. Best is trial 33 with value: 0.3125.
[I 2024-11-17 21:36:19,349] Trial 37 finished with value: 0.003403289846851957 and parameters: {'C': 1032.5965854757744, 'gamma': 0.0005207200738135311, 'kernel': 'linear'}. Best is trial 33 with value: 0.3125.
[I 2024-11-17 22:21:51,358] Trial 1 finished with value: 0.0 and parameters: {'C': 0.15518897538054796, 'gamma': 0.003143884090631424, 'kern

In [ ]:
  pipeline = Pipeline([
      #('outlier_remover', outlier_remover),
      ('preprocessor', preprocessor_knn),
      ('knn', KNeighborsClassifier())
  ])

  k = 2
  weights = 'distance'
  metric = 'manhattan'
  pipeline.set_params(knn__n_neighbors=k, knn__weights=weights, knn__metric=metric)
  pipeline.fit(x_train, y_train)

  y_pred = pipeline.predict(x_temp)
  score = f1_score(y_temp, y_pred)
  print(score)
  print(classification_report(y_temp, y_pred))




In [ ]:
print(score)

#TO DO
estudar separação de teste validação e treino

adicionar PCA a partir daqui, mas treinando com toda a base

Fazer for 30 vezes

In [ ]:
pca = PCA(n_components=5)
pca.fit(x_train)
X_train_pca = pca.fit_transform(x_train)
X_test_pca = pca.fit_transform(x_test)

X_train_pca = pd.DataFrame(X_train_pca)
X_test_pca = pd.DataFrame(X_test_pca)

In [ ]:

  # pipeline = Pipeline([
  #     #('outlier_remover', outlier_remover),
  #     ('preprocessor', preprocessor_knn),
  #     ('knn', KNeighborsClassifier())
  # ])

  k = 2
  weights = 'distance'
  metric = 'manhattan'
  # pipeline.set_params(knn__n_neighbors=k, knn__weights=weights, knn__metric=metric)
  # pipeline.fit(X_train_pca, y_train)
  model = KNeighborsClassifier(n_neighbors=k, weights=weights, metric=metric)
  model.fit(X_train_pca, y_train)
  y_pred = model.predict(X_test_pca)
  score = f1_score(y_test, y_pred)

  print(score)
  print(classification_report(y_test, y_pred))


In [ ]:
score = np.zeros(30)
acc = np.zeros(30)
rec = np.zeros(30)
prec = np.zeros(30)

for i in range(30):
  np.random.shuffle(df.values)
  x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)

  pipeline = Pipeline([
      #('outlier_remover', outlier_remover),
      ('preprocessor', preprocessor_knn),
      ('knn', KNeighborsClassifier())
  ])

  pipeline.set_params(knn__n_neighbors=k, knn__weights=weights, knn__metric=metric)
  pipeline.fit(x_train, y_train)
  y_pred = pipeline.predict(x_temp)
  score[i] = f1_score(y_temp, y_pred)
  acc[i] = accuracy_score(y_temp, y_pred)
  rec[i] = recall_score(y_temp, y_pred)
  prec[i] = precision_score(y_temp, y_pred)

print("F1_SCORE: \n")
print(f" Média -> {score.mean()}")
print(f" Desvio Padrão -> {score.std()}")
print("\nACCURACY: \n")
print(f" Média -> {acc.mean()}")
print(f" Desvio Padrão -> {acc.std()}")
print("\nRECALL: \n")
print(f" Média -> {rec.mean()}")
print(f" Desvio Padrão -> {rec.std()}")
print("\nPRECISION: \n")
print(f" Média -> {prec.mean()}")
print(f" Desvio Padrão -> {prec.std()}")
